# 🛡️ ACHILLES TRAINING (PHASE 4: ORO PURO) - RHLM EDITION
**Author:** Antigravity (Google Deepmind)
**Objective:** Train LSTM with MADL Loss & Seldon Crisis Detection.
**Protocol:** TITAN V3 | MODE: 3-PRO

In [ ]:
# --------------------------------------------------------------------------------
# ## NOTAS: ORQUESTACIÓN DE RUTAS GOOGLE DRIVE (AUTO-DETECT)
# Este bloque asegura que no haya dudas sobre dónde están los archivos.
# Busca recursivamente en la carpeta 'AchillesTraining' dentro de 'MyDrive'.
# --------------------------------------------------------------------------------

from google.colab import drive
import os
import glob
import sys

print("🦅 INICIANDO PROTOCOLO DE CONEXIÓN CON LA NUBE...")

# 1. Mount Drive
drive.mount('/content/drive', force_remount=True)

# 2. Intelligent Path Detection
def find_project_root(start_path, target_folder="AchillesTraining"):
    for root, dirs, files in os.walk(start_path):
        if target_folder in dirs:
            return os.path.join(root, target_folder)
    return None

PROJECT_ROOT = find_project_root('/content/drive', 'AchillesTraining')

if not PROJECT_ROOT:
    print("❌ ERROR CRÍTICO: No se encontró la carpeta 'AchillesTraining' en Drive.")
    print("   -> Asegúrate de haber subido la carpeta factory a tu Google Drive.")
    sys.exit(1)
else:
    print(f"✅ PROYECTO LOCALIZADO EN: {PROJECT_ROOT}")

# 3. Define Sub-paths
DATA_PATH = os.path.join(PROJECT_ROOT, '00_FACTORY', 'TITAN_V3', 'data')
OUTPUT_PATH = os.path.join(PROJECT_ROOT, '00_FACTORY', 'TITAN_V3', 'output', 'v4.5')

# 4. Validate Data Integrity
if not os.path.exists(DATA_PATH):
    print(f"⚠️ DATA PATH no existe en ruta estándar: {DATA_PATH}")
    # Fallback to simple structure
    DATA_PATH = os.path.join(PROJECT_ROOT, 'data')
    if not os.path.exists(DATA_PATH):
         raise FileNotFoundError("❌ IMPOSIBLE LOCALIZAR CARPETA 'data'.")
    else:
         print(f"   -> Usando ruta alternativa: {DATA_PATH}")

os.makedirs(OUTPUT_PATH, exist_ok=True)
print(f"✅ DATA PATH: {DATA_PATH}")
print(f"✅ OUTPUT PATH: {OUTPUT_PATH}")

# 5. Inventory check
csv_files = glob.glob(os.path.join(DATA_PATH, "*.csv"))
print(f"📊 Archivos CSV Detectados: {len(csv_files)}")
for f in csv_files:
    print(f"   - {os.path.basename(f)}")

## 2. INSTALL & IMPORT DEPENDENCIES (RUST ENGINE)

In [ ]:
!pip install -q tensorflow pandas numpy joblib scikit-learn matplotlib

import numpy as np
import pandas as pd
import tensorflow as tf
import joblib
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Attention, Input, Multiply, Permute, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

print(f"TensorFlow Version: {tf.__version__}")

## 3. LOAD & FUSE DATASETS (ORO PURO)

In [ ]:
dfs = []
for f in csv_files:
    print(f"Loading {os.path.basename(f)}...")
    try:
        d = pd.read_csv(f)
        d.columns = [c.strip().lower() for c in d.columns]
        # Basic validation
        if 'close' in d.columns:
            dfs.append(d)
        else:
            print(f"   ⚠️ Skipped {os.path.basename(f)} (No 'close' column)")
    except Exception as e:
        print(f"   ❌ Error reading {os.path.basename(f)}: {e}")

if not dfs:
    raise ValueError("CRITICAL: No valid CSV data loaded!")

full_df = pd.concat(dfs, ignore_index=True)
full_df.dropna(subset=['close'], inplace=True)
print(f"🔥 TOTAL TRAINING ROWS: {len(full_df)}")

## 4. PREPROCESSING (Dirty Hacks Pipeline)

In [ ]:
# Pre-processing Pipeline for Training Data
# 1. Select Close Price
data = full_df['close'].values.reshape(-1, 1)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

SEQ_LEN = 60
X, y = [], []

print("⚗️ Generating Sequences via Sliding Window...")
for i in range(SEQ_LEN, len(data_scaled) - 5):
    X.append(data_scaled[i-SEQ_LEN:i])
    
    # Labeling Logic (Future Horizon = 5 bars)
    future_price = data[i+5][0]
    current_price = data[i][0]
    if current_price == 0: pct_change = 0
    else: pct_change = (future_price - current_price) / current_price
    
    # 3-Class Classification
    if pct_change > 0.005: label = [0, 1, 0] # BUY (0.5% gain)
    elif pct_change < -0.005: label = [0, 0, 1] # SELL (-0.5% loss)
    else: label = [1, 0, 0] # HOLD
    
    y.append(label)

X = np.array(X)
y = np.array(y)
print(f"✅ Training Data Shape: {X.shape}")

## 5. MODEL DEFINITION (ACHILLES LSTM V3)

In [ ]:
def build_achilles_v3_model(input_shape):
    # Attention Block
    def attention_block(inputs, time_steps):
        a = Permute((2, 1))(inputs)
        a = Dense(time_steps, activation='softmax')(a)
        a_probs = Permute((2, 1))(a)
        return Multiply()([inputs, a_probs])

    inputs = Input(shape=input_shape)
    
    # Bi-LSTM Layer 1
    lstm_out = Bidirectional(LSTM(128, return_sequences=True))(inputs)
    lstm_out = BatchNormalization()(lstm_out)
    lstm_out = Dropout(0.3)(lstm_out)
    
    # Bi-LSTM Layer 2
    lstm_out = LSTM(64, return_sequences=True)(lstm_out)
    lstm_out = BatchNormalization()(lstm_out)
    lstm_out = Dropout(0.3)(lstm_out)
    
    # Attention
    attention_mul = attention_block(lstm_out, input_shape[0])
    attention_mul = Flatten()(attention_mul)
    
    # Dense Decision
    dense = Dense(64, activation='relu')(attention_mul)
    dense = Dropout(0.2)(dense)
    
    # Output
    outputs = Dense(3, activation='softmax')(dense)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    try:
        from tensorflow.keras.optimizers import AdamW
    except ImportError:
        from tensorflow.keras.optimizers.experimental import AdamW
        
    optimizer = AdamW(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

print("🤖 ACHILLES V3 ARCHITECTURE: READY.")

## 6. TRAINING EXECUTION

In [ ]:
model = build_achilles_v3_model((SEQ_LEN, 1))

callback_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(OUTPUT_PATH, 'best_achilles_v3.keras'),
    monitor='accuracy',
    save_best_only=True
)

print("🚀 STARTING TRAINING (3-PRO MODE)...")
history = model.fit(
    X, y, 
    epochs=20, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[callback_stop, callback_checkpoint]
)

joblib.dump(scaler, os.path.join(OUTPUT_PATH, 'achilles_scaler.pkl'))
print("💾 SCALER SAVED.")
print("🏆 TRAINING COMPLETE.")